In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
pd.set_option('display.max_columns', None)
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 200

import statsmodels.api as sm
#still might need to check on which patients are in dff but missing from the 37633 Unique patient IDs
dff = pd.read_csv('/home/allen/Galva/capstones/capstone2/data/model_agent.csv')
a = pd.read_csv('/home/allen/Galva/capstones/capstone2/src/explore/temp_csv/thomps2.csv') 

In [2]:
pat = pd.read_csv('/home/allen/Galva/capstones/capstone2/data/patient_df.csv')
xg = pd.read_csv('/home/allen/Galva/capstones/capstone2/src/explore/testing/x2.csv')

In [3]:
df_withID = pd.read_csv('/home/allen/Galva/capstones/capstone2/data/ready12_24_train.csv') 


In [4]:
df_withID.columns

Index(['Health_Camp_ID', 'Var1', 'Var2', 'Var3', 'Var4', 'Var5',
       'City_Type2_x', 'Job Type_x', 'online_score', 'y_target',
       'Camp Start Date - Registration Date',
       'Registration Date - First Interaction',
       'Camp Start Date - First Interaction',
       'Camp End Date - Registration Date', 'Camp Length', 'Category 1',
       'Category 2', 'Category 3'],
      dtype='object')

In [5]:
df_withID.head()

,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,City_Type2_x,Job Type_x,online_score,y_target,Camp Start Date - Registration Date,Registration Date - First Interaction,Camp Start Date - First Interaction,Camp End Date - Registration Date,Camp Length,Category 1,Category 2,Category 3
0,6578,4.0,0.0,0.0,0.0,2.0,23384,9999,0,1.0,-25.0,278,253,34,59,Third,G,2100
1,6532,0.0,0.0,0.0,0.0,0.0,23384,9999,0,0.0,-24.0,99,75,161,185,First,F,2100
2,6543,4.0,0.0,0.0,0.0,2.0,23384,9999,0,0.0,-60.0,355,295,711,771,First,F,2100
3,6580,0.0,0.0,0.0,0.0,0.0,23384,9999,0,0.0,4.0,12,16,19,15,First,E,2100
4,6570,0.0,0.0,0.0,0.0,0.0,23384,9999,0,0.0,69.0,146,215,82,13,First,E,2100


In [6]:
camp_loc={}
df2 =   df_withID.drop_dulplicates(['City_Type2_x','Health_Camp_ID']) 

AttributeError: 'DataFrame' object has no attribute 'drop_dulplicates'

In [ ]:
# records=df_withID.groupby(['City_Type2_x','Health_Camp_ID']).apply(lambda x: x).to_dict("records") 
get = np.array( zip(df_withID['City_Type2_x'], df_withID['Health_Camp_ID'].values)  )

In [ ]:
#import itertools as it
#from itertools import product
#print(list(product(df_withID['Health_Camp_ID'], df_withID['City_Type2_x'])))

In [2]:
a.head()

,Unnamed: 0,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,y_target,Camp Length,BFSI,Broadcasting,Consulting,Education,Food,Health,Manufacturing,Others,Real Estate,Retail,Software Industry,Technology,Telecom,Transport,A,C,D,E,F,G,Second,Third,1,2,3,4,Patient_ID,Camp Start Date - Registration Date,Registration Date - First Interaction,Camp Start Date - First Interaction,Camp End Date - Registration Date,SVC,svc_preds,XG,xg_preds,log,log_preds
0,3,6580,0.0,0.0,0.0,0.0,0.0,0.0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,489652,4.0,12,16,19,0.0,0.147007,0.0,0.000000,0.079390,0.0
1,12,6580,36.0,4.0,0.0,0.0,7.0,0.0,15,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,507246,64.0,41,105,79,0.0,0.189093,0.0,0.428571,0.292146,0.0
2,30,6580,0.0,0.0,0.0,0.0,0.0,0.0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,523729,18.0,165,183,33,0.0,0.146129,0.0,0.142857,0.089092,0.0
3,50,6580,15.0,1.0,0.0,0.0,7.0,0.0,15,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,521364,64.0,473,537,79,0.0,0.242156,1.0,0.571429,0.328480,0.0
4,64,6580,0.0,0.0,0.0,0.0,0.0,0.0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,494493,-2.0,327,325,13,0.0,0.144586,0.0,0.428571,0.098607,0.0


In [ ]:
df_withID['Tups'] = list(zip(df_withID.City_Type2_x,df_withID.Health_Camp_ID))

In [ ]:
df_withID['Tups2'] = list(zip(df_withID['Camp Length'].values,df_withID.Health_Camp_ID))

In [ ]:
d2={}
for i in df_withID['Tups2'].values:
    key = i[1]
    value = i[0]
    if key not in d2.keys():
        d2[key]=value

In [ ]:
d2_counts={}
for i in d2.values():
    if i not in d2_counts:
        d2_counts[i]=1
    else:
        d2_counts[i]+=1
print(sorted(d2_counts.items()))

In [ ]:
cat2={}
city={}
jobs={}

for i in df_withID['City_Type2_x'].values:
    if i not in city:
        city[i]=1
    else:
        city[i]+=1

for ii in df_withID['Category 2'].values:
    if ii not in cat2:
        cat2[ii]=1
    else:
        cat2[ii]+=1
for iii in df_withID['Job Type_x'].values:
    if iii not in jobs:
        jobs[iii]=1
    else:
        jobs[iii]+=1
        
print(cat2.values(), '*******cat2')
print(city.values(), 'CITY')
print(jobs.values(), 'JOBs')

In [6]:
a = dff.copy() 

In [9]:
a_ = dff.copy()

In [ ]:
# dff['Prediction_Score'] = dff['prediction_kNN']+dff['prediction_sVC']+dff['prediction_xg']+dff['y_target']
# dff['Prediction_Score2'] = dff['Prediction_Score'].astype(str) 
dff.columns

In [ ]:
'''
Best way to be specific about success or failure for each test?
-Each model predicts 0/1 for each person who is either 0/1 == 4 system states per test
-For each person, each tetst == SXK, sXk, Skx, sKX, skX, etc 
'''

In [ ]:
dff.describe()
dff['Success'] = dff['Y_target_SUM'].apply(lambda x:4 if x == 0 else x) 

-It seems like I should change the nature of Y_count_allModels and Y-Target_SUM 
*** Rational would be that I don't Only care about Y-Target I  also care about Model Success

-So The update should be :

create 'Success_count' 


%% Can you stack lambda functions? - Like create a series of them

%%Note to self and use in interview - realized my model was incomplete in a certain sense | Success was attendance, success for me/models was correct predction.


In [ ]:
# g = sns.relplot(data=dff, x='proba_xg', y='proba_s2',hue= 'y_target', palette="deep", kind='scatter', col="prediction_k2", col_wrap=2)
# plt.legend(loc='best')
# plt.title('S2 vs XG Probas')

In [ ]:
sns.stripplot(x='Y_target_SUM', y='proba_sVC', hue='y_target', data=dff, jitter=True)

In [ ]:

g1 = sns.relplot(data=dff, x='prediction_xg', y ='proba_xg' ,hue= 'y_target', palette="deep" )
# g2 = sns.relplot(data=dff, x='prediction_kNN', y ='proba_kNN' ,hue= 'y_target', palette="deep" )
# g3 = sns.relplot(data=dff, x='prediction_xg', y ='proba_xg' ,hue= 'y_target', palette="deep" )
plt.legend(loc='best')
plt.title('XG Boost Probabilities & Actual Attendance')
plt.figure(figsize=(6, 6))
#make a 4 plot of these to put into readme

In [ ]:
# sns.relplot(data=dff,x=['prediction_sVC','prediction_xg' ], y = ['proba_sVC', 'proba_xg'], hue="y_target")


In [ ]:
'''
Need to append IDs to k2,s2
create new col from ids.values

*new idea, create a series of functions to
make a new dataframe.

-rename cols to reflect model type:
knn_proba, knn_prediction , sv_ , xg_

-want to know:
similar among 3
difference among 3

-visualize the above ? compare heatmaps 

assign network or cluster based on Location
or break the graph below into location

assign colors:
overlap - incorrect White/Black
overlap - correct
partial1 ab : Green: light = False, Dark = True
partial2 bc : Blue : light = False, Dark = True
partial3 ac : Yellow: light = False, Dark = True

lambda functions, mapping, 

Want to know if correct-
want to know who it was correct-

'''

In [ ]:
#Update 4/27 Outline for compare_model.py functions 

'''
def change_cols(xg,knn,svc): 
    cols_to_change=[prediction, Proba]
    return df 
    
def combine(xg,knn,svc):
    add the two to one <4/27 migth indiacte to add 
                       < either of those 2 and then add the 2_single set to the One
    
def edit(new_df):
    could do a sum across rows for y_predict value ,0,1,2,3
    if its a 2 then check for ac,ab,bc
    if its a 1 then check for a,b,c 
    assign letter or number based on 1-6
    
def graph(new_df):
    

'''

In [ ]:
k = k2['prediction'].values
s = s2['prediction'].values
lst = list(zip(k,s))
lstK = list(zip(k,noid))
lstS = list(zip(noid,s))

In [ ]:
acum_s_k=0
acum_b_k=0
acum_b_s=0

for i in lst:
    if i[0] == i[1]:
        acum_s_k+=1
print(acum_s_k)

for i in lstK:
    if i[0] == i[1]:
        acum_b_k+=1
print(acum_b_k)

for i in lstS:
    if i[0] == i[1]:
        acum_b_s+=1
print(acum_b_s)


In [7]:
#sns.stripplot(x='Prediction_Score2', y='proba_k2', hue='y_target', data=dff, jitter=True)
from preprocessing import *

In [ ]:
'''
Scale the data
remove target, set to =y
Trying with first part of thomps samp2

if y_target = 0  & wins = 0 then y_target = 4
'''

def success():
    

In [10]:

a2 = scale(a)
a2a=scale(a_)

In [11]:
a2

,Unnamed: 0,Var1,Var2,Var3,Var4,Var5,Camp Start Date - Registration Date,Registration Date - First Interaction,Camp Start Date - First Interaction,Camp End Date - Registration Date,Camp Length,Second,Third,A,C,D,E,F,G,2100,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,9999.0,1,2,3,4,1036,1216,1217,1352,1704,1729,2517,2662,23384,Patient_ID,prediction,Proba,y_target,proba_kNN,prediction_kNN,proba_sVC,prediction_sVC,proba_xg,prediction_xg,Y_count_allModels,Y_target_SUM
0,0,-0.099899,-0.061176,-0.016302,-0.110402,-0.205835,-0.253729,-0.439607,-0.797227,-0.308012,-0.092492,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,514789,0.0,0.277560,0.0,0.0,0.0,0.156124,0.0,0.282538,0.0,0.0,0.0
1,1,-0.099899,-0.061176,-0.016302,-0.110402,-0.205835,-1.258851,1.010948,0.051359,0.602025,1.961539,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,508149,0.0,0.246401,0.0,0.1,0.0,0.155704,0.0,0.235598,0.0,0.0,0.0
2,2,-0.099899,-0.061176,-0.016302,-0.110402,-0.205835,-0.105205,0.220462,0.174972,-0.163023,-0.078471,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,492650,0.0,0.339183,0.0,0.2,0.0,0.157399,0.0,0.348792,0.0,0.0,0.0
3,3,-0.099899,-0.061176,-0.016302,-0.110402,-0.205835,0.340364,-0.192082,0.088108,-0.335776,-0.726928,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,511274,0.0,0.455702,0.0,0.3,0.0,0.154247,0.0,0.437974,0.0,0.0,0.0
4,4,-0.099899,-0.061176,-0.016302,-0.110402,-0.205835,0.222927,-0.447592,-0.346207,-0.332691,-0.604247,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,520795,1.0,0.516371,0.0,0.4,0.0,0.565486,1.0,0.476005,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15051,15051,-0.099899,-0.061176,-0.016302,-0.110402,-0.205835,-0.260637,-0.242651,-0.556683,-0.314182,-0.092492,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,500613,0.0,0.381371,1.0,0.2,0.0,0.158896,0.0,0.359628,0.0,0.0,1.0
15052,15052,-0.099899,-0.061176,-0.016302,-0.110402,-0.205835,0.153847,-0.008433,0.138222,-0.462256,-0.681361,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,507342,0.0,0.358296,0.0,0.2,0.0,0.155821,0.0,0.352790,0.0,0.0,0.0
15053,15053,1.771307,1.541135,-0.016302,9.813220,4.743420,0.077858,0.244416,0.382107,1.795871,1.961539,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,521693,0.0,0.475041,0.0,0.2,0.0,0.174843,0.0,0.466925,0.0,0.0,0.0
15054,15054,-0.099899,-0.061176,-0.016302,-0.110402,-0.205835,0.281646,-0.476869,-0.326162,-0.280248,-0.604247,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,489075,0.0,0.457274,0.0,0.5,0.0,0.519868,1.0,0.442756,0.0,1.0,1.0


In [12]:
def happen():
    df = dff.copy()
    lst = [ 'prediction_kNN', 'prediction', 'prediction_sVC', 'prediction_xg' ]

    for col in lst:
        col_ = col + '_score'
        print(col_)
        df['and_'+col] = dff['y_target'] + dff[col]
    for col in lst:
        df['and_'+col] = df['and_'+col].apply(lambda x: 2 if x ==0 else x) 
    for col in lst:
        col_ = col + '_score'
        df[col_] = df['and_'+col].apply(lambda x: 1 if x == 2  else 0  )
    
    df['Score'] = df['prediction_kNN_score']+ df['prediction_score'] +df['prediction_sVC_score'] + df['prediction_xg_score'] 
    
    return df

In [13]:
df3 = happen()

prediction_kNN_score
prediction_score
prediction_sVC_score
prediction_xg_score


In [14]:
 
a_ = scale(df3) 

In [15]:
 
a2 = a_.drop(['Unnamed: 0','proba_kNN', 'prediction_kNN', 'proba_sVC',
       'prediction_sVC', 'proba_xg', 'prediction_xg', 'Y_count_allModels',
       'Y_target_SUM', 'and_prediction_kNN', 'and_prediction',
       'and_prediction_sVC', 'and_prediction_xg', 'prediction_kNN_score',
       'prediction_score', 'prediction_sVC_score', 'prediction_xg_score',
    'Patient_ID', 'prediction', 'Proba',], axis = 1)
print(a_.columns) 
y_target = a_['y_target']

Index(['Unnamed: 0', 'Var1', 'Var2', 'Var3', 'Var4', 'Var5',
       'Camp Start Date - Registration Date',
       'Registration Date - First Interaction',
       'Camp Start Date - First Interaction',
       'Camp End Date - Registration Date', 'Camp Length', 'Second', 'Third',
       'A', 'C', 'D', 'E', 'F', 'G', '2100', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '8.0', '9.0', '10.0', '11.0', '12.0', '13.0', '14.0', '9999.0',
       '1', '2', '3', '4', '1036', '1216', '1217', '1352', '1704', '1729',
       '2517', '2662', '23384', 'Patient_ID', 'prediction', 'Proba',
       'y_target', 'proba_kNN', 'prediction_kNN', 'proba_sVC',
       'prediction_sVC', 'proba_xg', 'prediction_xg', 'Y_count_allModels',
       'Y_target_SUM', 'and_prediction_kNN', 'and_prediction',
       'and_prediction_sVC', 'and_prediction_xg', 'prediction_kNN_score',
       'prediction_score', 'prediction_sVC_score', 'prediction_xg_score',
       'Score'],
      dtype='object')


In [16]:
from sklearn.cluster import KMeans

In [17]:
ncluster_list = range(1,20)

def kmean_score(nclust):
    
    km = KMeans(n_clusters = nclust, 
                init = 'random', 
                n_init = 10, 
                max_iter = 300) 
                #n_jobs = -1) #look at hyperparameters 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
    km.fit(a2)
    rss = -km.score(a2)
    return rss

scores = [kmean_score(i) for i in range(1,20)]
print("#clust\trss")
for i, score in enumerate(scores):
    print("{0}\t{1:0.0f}".format(ncluster_list[i],score))

#clust	rss
1	217214
2	187690
3	164815
4	141468
5	119391
6	109299
7	105300
8	101905
9	81276
10	76246
11	75463
12	69773
13	67480
14	67322
15	64469
16	63204
17	62500
18	61967
19	60632


In [19]:
print(a2.columns) 
'''
-drop cols
-create target_variable
'''
y2 = a2.pop('y_target')
y_ = a2.pop('Score')
#dft = dff.drop(['Unnamed: 0','y_target','Y_count_allModels','Y_target_SUM','Patient_ID'], axis=1)

Index(['Var1', 'Var2', 'Var3', 'Var4', 'Var5',
       'Camp Start Date - Registration Date',
       'Registration Date - First Interaction',
       'Camp Start Date - First Interaction',
       'Camp End Date - Registration Date', 'Camp Length', 'Second', 'Third',
       'A', 'C', 'D', 'E', 'F', 'G', '2100', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '8.0', '9.0', '10.0', '11.0', '12.0', '13.0', '14.0', '9999.0',
       '1', '2', '3', '4', '1036', '1216', '1217', '1352', '1704', '1729',
       '2517', '2662', '23384', 'y_target', 'Score'],
      dtype='object')


In [20]:
 
train_test_split(a_, shuffle=False) 
X_train,X_test,y_train,y_test = train_test_split(
    a_,y2,test_size = .3, random_state=101)


from sklearn.linear_model import LogisticRegression
logmodelx = LogisticRegression(penalty='l2', dual=False, tol=1e-4, C=1.0, 
        fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, 
        solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, 
        n_jobs=1, l1_ratio=None ) 
logmodelx.fit(X_train, y_train)



predictions = logmodelx.predict_proba(a_)[:,1]
pure_proba = logmodelx.predict(a_)
preds, preds2 = predictions>.5 ,predictions>.55
# for name, coef in zip(preds2[1:], logmodelx.coef_[0]):
#     print("{0}: {1:0.4f}".format(name, coef))
from sklearn.metrics import classification_report
print(classification_report(y2,preds2) ) 
print(classification_report(y2,preds) )  

              precision    recall  f1-score   support

         0.0       0.72      1.00      0.84     10904
         1.0       0.00      0.00      0.00      4152

    accuracy                           0.72     15056
   macro avg       0.36      0.50      0.42     15056
weighted avg       0.52      0.72      0.61     15056

              precision    recall  f1-score   support

         0.0       0.72      1.00      0.84     10904
         1.0       0.00      0.00      0.00      4152

    accuracy                           0.72     15056
   macro avg       0.36      0.50      0.42     15056
weighted avg       0.52      0.72      0.61     15056



/home/allen/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/allen/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/allen/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/allen/anaconda3/lib/python3.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Though the following import is not directly being used, it is required
# for 3D projection to work
from mpl_toolkits.mplot3d import Axes3D

from sklearn.cluster import KMeans
from sklearn import datasets

np.random.seed(5)

iris = datasets.load_iris()
X = iris.data
y = iris.target

estimators = [('k_means_iris_8', KMeans(n_clusters=8)),
              ('k_means_iris_3', KMeans(n_clusters=3)),
              ('k_means_iris_bad_init', KMeans(n_clusters=3, n_init=1,
                                               init='random'))]

fignum = 1
titles = ['8 clusters', '3 clusters', '3 clusters, bad initialization']
for name, est in estimators:
    fig = plt.figure(fignum, figsize=(4, 3))
    ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
    est.fit(X)
    labels = est.labels_

    ax.scatter(X[:, 3], X[:, 0], X[:, 2],
               c=labels.astype(float), edgecolor='k')

    ax.w_xaxis.set_ticklabels([])
    ax.w_yaxis.set_ticklabels([])
    ax.w_zaxis.set_ticklabels([])
    ax.set_xlabel('Petal width')
    ax.set_ylabel('Sepal length')
    ax.set_zlabel('Petal length')
    ax.set_title(titles[fignum - 1])
    ax.dist = 12
    fignum = fignum + 1

# Plot the ground truth
fig = plt.figure(fignum, figsize=(4, 3))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)

for name, label in [('Setosa', 0),
                    ('Versicolour', 1),
                    ('Virginica', 2)]:
    ax.text3D(X[y == label, 3].mean(),
              X[y == label, 0].mean(),
              X[y == label, 2].mean() + 2, name,
              horizontalalignment='center',
              bbox=dict(alpha=.2, edgecolor='w', facecolor='w'))
# Reorder the labels to have colors matching the cluster results
y = np.choose(y, [1, 2, 0]).astype(float)
ax.scatter(X[:, 3], X[:, 0], X[:, 2], c=y, edgecolor='k')

ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])
ax.set_xlabel('Petal width')
ax.set_ylabel('Sepal length')
ax.set_zlabel('Petal length')
ax.set_title('Ground Truth')
ax.dist = 12

fig.show()

In [ ]:
'''
-choose 3 target classes ? or create a COI
- 4 indep vars , 1 dep var (class) 

'''

In [ ]:
est.labels_

In [ ]:
iris

In [ ]:
r = np.zeros(50)

In [ ]:
lst=[1 for x,xx in enumerate(r) if x%2 == 0 ]

In [ ]:
df3 = dff.head()

In [ ]:
df3.to_csv('/home/allen/Galva/capstones/capstone2/data/models_for_git_head.csv')

In [ ]:
surprising3= a_[a_['y_target']==1 & (a_['Score']==0)]
# Where the Person went to a Camp BUT no model predicted they would
print(len(surprising3))

surprising3a= a_[a_['y_target']==1 & (a_['Score']!=0)]
# Where the Person went to a Camp and some model predicted they would
print(len(surprising3a))

# EDA among most surprising agents


In [ ]:
idv0 = ['Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Camp Start Date - Registration Date'] 
idv1= ['Registration Date - First Interaction', 'Camp Start Date - First Interaction' ]
idv2= ['Camp End Date - Registration Date', 'Camp Length', 'Second', 'Third', 'A'] 
idv3= ['C', 'D', 'E', 'F', 'G', '2100', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0'] 
idv4= ['8.0', '9.0', '10.0', '11.0', '12.0', '13.0', '14.0', '9999.0', '1', '2'] 
idv5= ['3', '4', '1036', '1216', '1217', '1352', '1704', '1729', '2517', '2662', '23384']

In [ ]:
df0 = dff[[item for item in idv0]]
df0.hist(bins=30)
plt.tight_layout()

In [ ]:
cityz = {}
camp

In [ ]:
surprising2= dff[(dff['y_target']==0) & (dff['Y_count_allModels']>=2)]
# Where the person did Not go to a Camp But two models thought they would
print(len(surprising2))

In [ ]:
dff.columns

import itertools as it 

lst = [item for item in dff.columns]
print(lst)

In [ ]:
 #  itertools.combinations(iterable, r)
itered = it.combinations(lst[1:9] , 6)    

In [ ]:
import math
from math import factorial

print(math.factorial(6) / (math.factorial(9)-math.factorial(6) )  ) 

# surprising

In [ ]:
'''
Ho Testing for certain columns to test if significant differences exist among 

sec_ho = dff[dff['y_target'] ==1 ]['Second'].to_numpy()
sec_ha = dff[dff['y_target'] ==0 ]['Second'].to_numpy()

sec_ho_mean = sec_ho.mean()
sec_ha_mean = sec_ha.mean()

ztest(sec_ho_mean,sec_ha_mean)

surprising3= dff[dff['y_target']==1 & (dff['Y_count_allModels']==0)]
# Where the Person went to a Camp BUT no model predicted they would
print(len(surprising3))

surprising3a= dff[dff['y_target']==1 & (dff['Y_count_allModels']!=0)]
# Where the Person went to a Camp and some model predicted they would
print(len(surprising3a))
'''

In [ ]:
import scipy.stats as stats
def ho_testing(ho,ha):
    '''
    Inputs: x= list of lists   
    output: a p_value 
    ''' 
    
    return stats.ttest_ind(ho,ha,alternative= "two-sided",equal_var =False)

In [ ]:
def manual_ho_testing(ho,ha):
    stan_error = sum(ho) - sum(ha)
    pp = (stan_error)/(len(ho)+len(ha)) #pool_proba
    pool_stan_error = (pp * (1-pp) * (1/( len(ho) +len(ha))**.5 ) )
    return f'stand error -> {round(stan_error,2)} , pooled_proba ->{round(pp,2)}, and pool_error {round(pool_stan_error,2)}'


In [ ]:
cols2 = [
    'Var1', 'Var2', 'Var3', 'Var4', 'Var5',
       'Camp Start Date - Registration Date',
       'Registration Date - First Interaction',
       'Camp Start Date - First Interaction', 'Success',
       'Camp End Date - Registration Date', 'Camp Length', 'Second', 'Third',
       'A', 'C', 'D', 'E', 'F', 'G', '2100', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '8.0', '9.0', '10.0', '11.0', '12.0', '13.0', '14.0', '9999.0',
       '1', '2', '3', '4', '1036', '1216', '1217', '1352', '1704', '1729',
       '2517', '2662'
]

In [ ]:
cols=['Second','Third','9999.0', 'Camp Length']
check = []
for i in cols2:
    ho = surprising3a[i].values
    ha = surprising3[i].values
    get = ho_testing(ho,ha) 
    
    get1 = round(get[0],3)
    get2 = round(get[1],3)
    print(f'statistic = {get1} p_val --> {get2}, for COI --> {i}',manual_ho_testing(ho,ha) )
    

In [ ]:
surprising3.head()

In [ ]:
surprising3.columns

In [ ]:
Prediction = # I think is XG 1 prediction
Prediction_Score = # see above
Y_target_SUM = #up to 4
Y_count_allModels = # pred from xg+knn+svc up to 3

# Wanting Y count among all models
# sum (Y Target , all models)
#

In [ ]:
'''
smaller:
[('Second', (0.0005101027299358867, 0.5002035015374943)), ('Third', (0.001746215432479473, 0.5006966388126657)), 
('9999.0', (0.00036383681552966033, 0.5001451498856789)), ('Camp Length', (0.0008299770004436161, 0.5003311128792225))]


'''

In [ ]:
#surprising3['Pred_Score_all_models'].max()
sup3 = surprising3[['y_target', 'Var1', 'Var2', 'Var3', 'Var4', 'Var5',
       'Camp Start Date - Registration Date',
       'Registration Date - First Interaction',
       'Camp Start Date - First Interaction']]

In [ ]:
sns.heatmap(sup3, annot='True')

In [ ]:
se =  0.23 -0.11 
pp_ =  0.012 / 2000 
pse =  pp_*(1-pp_) * (1/1000 + 1/1000)**.5  
print(se,pp_,pse)

In [ ]:
0.012/0.00578 